### Import Packages

In [ ]:
import os
import pyodbc
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};\
                       Server=lasr-sqldb-prd-im,17001;\
                       Database=IM_I_IDWBRRADM_S;\
                       Trusted_Connection=yes;')

In [3]:
sql_query = """
select
pfm.RAPORTFOLIOID,
XW.portfoliouid,
XW.aggrid,
pfm.postdt,
pfm.RTNPERIODTYPCD,
cg.InvestmentPortfolioTypeCode,
ia.InvestmentAccountUID,
ia.InvestmentAccountTypeCode,
ia.SeparateAccountTypeCode,
ia.FundAccountTypeCode,
ia.FundAccountSubTypeCode,
ia.FundAccountGroupTypeCode
from
(
select
RAPORTFOLIOID,
RTNPERIODENDDT as postdt,
RTNPERIODTYPCD,
RAMARKETCYCLEID
from IM_I_IDWBRRADM_S.dbo.RA_PORT_RTN
where CRNCYUID = 1 and RTNPCT is not NULL
) as pfm
  inner join
(
select
RAPORTFOLIOID,
RTNPERIODENDDT as postdt,
RTNPERIODTYPCD,
max(RAMARKETCYCLEID) as max_cycle
from IM_I_IDWBRRADM_S.dbo.RA_PORT_RTN
where CRNCYUID = 1 and RTNPCT is not NULL
group by RAPORTFOLIOID,RTNPERIODENDDT,RTNPERIODTYPCD
) as B
on pfm.RAPORTFOLIOID=B.RAPORTFOLIOID and pfm.postdt=B.postdt and pfm.RTNPERIODTYPCD=B.RTNPERIODTYPCD and pfm.RAMARKETCYCLEID=B.max_cycle 
left join
	(
	select
	RAPORTFOLIOID,
	PORTUID as portfoliouid,
	INVPORTID as aggrid
	from IM_I_IDWBRRADM_S.dbo.RA_PORTFOLIO
	--where PORTUID is not NULL
	) as XW
on pfm.RAPORTFOLIOID = XW.RAPORTFOLIOID
-- the labeling stuff here --
  left join 
     IM_I_PORTFOLIOPOS_S.dbo.cgportfolio as cg
  on XW.PortfolioUID=cg.PortfolioUID and pfm.postdt between  cg.dweffectivefromdate and cg.dweffectivetodate

  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AA as AA
  on cg.portfoliouid =  AA.AAPortfolioUID and pfm.postdt between  AA.dweffectivefromdate and AA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AA'
  left join
     IM_U_RESULTS_S.KPMG.KMPYAD_AR as AR
  on cg.portfoliouid =  AR.ARPortfolioUID and pfm.postdt between  AR.dweffectivefromdate and AR.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AR'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AT as AT
  on cg.portfoliouid =  AT.ATPortfolioUID and pfm.postdt between  AT.dweffectivefromdate and AT.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AT'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_RP as RP
  --on cg.portfoliouid =  RP.RPPortfolioUID and pfm.postdt between  RP.dweffectivefromdate and RP.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'RP'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_AM as AM
  on cg.portfoliouid =  AM.AMPortfolioUID and pfm.postdt between  AM.dweffectivefromdate and AM.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'AM'
  --the extra non-jasper ones
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_GA as GA
  on cg.portfoliouid =  GA.GAPortfolioUID and pfm.postdt between  GA.dweffectivefromdate and GA.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'GA'
  left join
       IM_U_RESULTS_S.KPMG.KMPYAD_ID as ID
  on cg.portfoliouid =  ID.IDPortfolioUID and pfm.postdt between  ID.dweffectivefromdate and ID.dweffectivetodate and cg.InvestmentPortfolioTypeCode = 'ID'
  -----acct_mgr table-----
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_GR as GR
  --on XW.aggrid =  GR.RPPCXGRPAGGRID and pfm.postdt between  GR.STARTDT and GR.ENDDT and cg.InvestmentPortfolioTypeCode = 'GR'
  --left join
  --     IM_U_RESULTS_S.KPMG.KMPYAD_SR as SR
  --on XW.aggrid =  SR.RPPCXSPRGRPAGGRID and pfm.postdt between  SR.STARTDT and SR.ENDDT and cg.InvestmentPortfolioTypeCode = 'SR'
  -- finally join to investment account
  left join
     IM_I_ACCTPOSTRANS_S.dbo.InvestmentAccount as ia
	       ---case it out---
  on ia.InvestmentAccountUID = case when cg.InvestmentPortfolioTypeCode='AA' THEN AA.InvestmentAccountUID
                                    when cg.InvestmentPortfolioTypeCode='AR' THEN AR.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AT' THEN AT.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode='RP' THEN RP.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='AM' THEN AM.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='GA' THEN GA.InvestmentAccountUID
									when cg.InvestmentPortfolioTypeCode='ID' THEN ID.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'GR' THEN GR.InvestmentAccountUID
									--when cg.InvestmentPortfolioTypeCode  = 'SR' THEN SR.InvestmentAccountUID
								end
  and pfm.postdt between ia.dweffectivefromdate and ia.dweffectivetodate
"""
# Save the data
data = pd.read_sql(sql_query, cnxn)
#close connection
cnxn.close()

In [4]:
print(data.shape)
data.head()

(17354336, 12)


,RAPORTFOLIOID,portfoliouid,aggrid,postdt,RTNPERIODTYPCD,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode
0,6471,157698.0,127314947.0,2017-04-28,MCPC,AM,16000016.0,FUND,None,MF,AF,NG
1,78479,145208.0,117027677.0,1997-08-29,1MO,GM,NaN,None,None,None,None,None
2,103549,22699.0,18170772.0,2006-09-29,1MO,AM,176100664.0,AG,None,None,None,None
3,9874,144179.0,116082042.0,2000-06-30,3MO,AA,NaN,None,None,None,None,None
4,247675,201958.0,165012037.0,2017-09-29,LIFEPRD,AM,5000005.0,FUND,None,MF,AF,NG


In [5]:
data['postdt_new'] = data['postdt'].apply(lambda dt: dt.replace(day=1))
data.drop(['postdt'], axis=1, inplace=True)
data.head()

,RAPORTFOLIOID,portfoliouid,aggrid,postdt,RTNPERIODTYPCD,InvestmentPortfolioTypeCode,InvestmentAccountUID,InvestmentAccountTypeCode,SeparateAccountTypeCode,FundAccountTypeCode,FundAccountSubTypeCode,FundAccountGroupTypeCode,postdt_new
0,6471,157698.0,127314947.0,2017-04-28,MCPC,AM,16000016.0,FUND,None,MF,AF,NG,2017-04-01
1,78479,145208.0,117027677.0,1997-08-29,1MO,GM,NaN,None,None,None,None,None,1997-08-01
2,103549,22699.0,18170772.0,2006-09-29,1MO,AM,176100664.0,AG,None,None,None,None,2006-09-01
3,9874,144179.0,116082042.0,2000-06-30,3MO,AA,NaN,None,None,None,None,None,2000-06-01
4,247675,201958.0,165012037.0,2017-09-29,LIFEPRD,AM,5000005.0,FUND,None,MF,AF,NG,2017-09-01


In [7]:
#check no duplicate records --there are none
print(data.drop_duplicates(subset=['RAPORTFOLIOID', 'postdt_new', 'RTNPERIODTYPCD'], keep='first', inplace=False).count())

RAPORTFOLIOID                  17353296
portfoliouid                   17351857
aggrid                         15440003
RTNPERIODTYPCD                 17353296
InvestmentPortfolioTypeCode    15436704
InvestmentAccountUID            6209960
InvestmentAccountTypeCode       6209960
SeparateAccountTypeCode          120928
FundAccountTypeCode              881221
FundAccountSubTypeCode           573228
FundAccountGroupTypeCode         804212
postdt_new                     17353296
dtype: int64


In [8]:
data.rename(columns={'postdt_new': 'postdt', 'RTNPERIODTYPCD': 'rtntyp', 'InvestmentPortfolioTypeCode': 'Portype', 
                    'InvestmentAccountTypeCode': 'lab1', 'SeparateAccountTypeCode': 'lab2', 'FundAccountTypeCode': 'lab3',
                    'FundAccountSubTypeCode': 'lab4', 'FundAccountGroupTypeCode': 'lab5'}, inplace=True)
data.drop(['portfoliouid','aggrid','InvestmentAccountUID'], axis=1, inplace=True)
print(data.shape)
data.head()

(17354336, 9)


,RAPORTFOLIOID,rtntyp,Portype,lab1,lab2,lab3,lab4,lab5,postdt
0,6471,MCPC,AM,FUND,None,MF,AF,NG,2017-04-01
1,78479,1MO,GM,None,None,None,None,None,1997-08-01
2,103549,1MO,AM,AG,None,None,None,None,2006-09-01
3,9874,3MO,AA,None,None,None,None,None,2000-06-01
4,247675,LIFEPRD,AM,FUND,None,MF,AF,NG,2017-09-01


In [10]:
df_final_filled = data.fillna('N/A')

In [11]:
df_final_filled.head()

,RAPORTFOLIOID,rtntyp,Portype,lab1,lab2,lab3,lab4,lab5,postdt
0,6471,MCPC,AM,FUND,N/A,MF,AF,NG,2017-04-01
1,78479,1MO,GM,N/A,N/A,N/A,N/A,N/A,1997-08-01
2,103549,1MO,AM,AG,N/A,N/A,N/A,N/A,2006-09-01
3,9874,3MO,AA,N/A,N/A,N/A,N/A,N/A,2000-06-01
4,247675,LIFEPRD,AM,FUND,N/A,MF,AF,NG,2017-09-01


In [12]:
df_summary_out = df_final_filled.groupby(['postdt','rtntyp', 'Portype', 'lab1', 'lab2', 'lab3', 'lab4', 'lab5'])['RAPORTFOLIOID'].count().reset_index()
df_summary_out.rename(columns={'RAPORTFOLIOID': 'raport_cnt'}, inplace=True)
print(df_summary_out.shape)
df_summary_out.head()

(222947, 9)


,postdt,rtntyp,Portype,lab1,lab2,lab3,lab4,lab5,raport_cnt
0,1934-01-01,1MO,FL,N/A,N/A,N/A,N/A,N/A,1
1,1934-01-01,LIFE,FL,N/A,N/A,N/A,N/A,N/A,1
2,1934-01-01,LIFEPRD,FL,N/A,N/A,N/A,N/A,N/A,1
3,1934-01-01,YTD,FL,N/A,N/A,N/A,N/A,N/A,1
4,1934-02-01,1MO,FL,N/A,N/A,N/A,N/A,N/A,1


In [16]:
df_summary_out.to_csv('RA_PORT_RTN_w_Labels_Summarized_20181008.csv')

In [14]:
df_summary_out.raport_cnt.sum()

17354336